In [ ]:
# Standard libraries
import json
import math
import os
import random
from typing import (Any, Dict, List, Tuple)

# Third party libraries
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io.wavfile
import scipy.signal
import sklearn
import sklearn.tree


print('IPython.__version__ = %s' % IPython.__version__)
print('matplotlib.__version__ = %s' % matplotlib.__version__)
print('numpy.__version__ = %s' % np.__version__)
print('pandas.__version__ = %s' % pd.__version__)
print('scipy.__version__ = %s' % scipy.__version__)
print('sklearn.__version__ = %s' % sklearn.__version__)

print('\nseeding random with %d\n' % int("9fe3ddf4da76a6", 16))
random.seed(int("9fe3ddf4da76a6", 16))

with open('/proc/%d/status' % os.getpid(), 'rb') as f:
    print(f.read().decode('utf-8'))

In [ ]:
# Raw audio: ../audios/*.wav
# Labels: ../outputs/*.json --> ../labels/*.tsv
youtube_ids = set([filename.split('.')[0] for filename in os.listdir('../audios/')])
training_files = set([
    'NrgmdOz227I', # Yesterday
])
eval_files = set([
    'wr2sVPTacTE', # kelly, wife or dog
    'TjUXr560Gu0', # dhoom taana
])

# sampling? first-N-seconds? last-N-seconds? specific time-window?
# lazy loading? mono-channel?

def sampleData(rate: int, data: np.ndarray, limit_secs: int) -> np.ndarray:
    limit = rate * limit_secs  # rate (samples/second) * seconds -> num samples
    return data[:limit, 0]


def labelsFromUtterances(utterances: List[Dict[str, Any]], windows_per_second: int, n_rows: int) -> Tuple[int, np.ndarray]:
    """return: np.ndarray[ndtype=intish, shape=[n_rows]]"""
    positive_examples = 0
    is_talking = np.zeros(n_rows, dtype=np.int8)
    for item in utterances:
        # math.ceil rounds up to the latest millisecond for labeling
        start_i = int(math.ceil(item['start'] * windows_per_second))
        end_i = int(math.ceil(item['end'] * windows_per_second))
        for i in range(start_i, min(end_i, n_rows)):
            is_talking[i] = 1
            positive_examples += 1
    return positive_examples, is_talking


# TODO use a dataclass instead of a Dict
def readData(video_id: str, limit: int) -> Dict[str, Any]:
    # dtype should be np.dtype('int16')
    rate, all_data = scipy.io.wavfile.read('../audios/%s.wav' % video_id)
    data = sampleData(rate, all_data, limit)
    
    # try wrapping in `int(2 ** math.ceil(math.log(.., 2)))`
    window_size = int(rate) // 100
    step_size = window_size // 2
    # we want windows_per_second to be 200
    windows_per_second = int(rate) // step_size
    _freqs, _times, spectro = scipy.signal.stft(
        data,
        rate,
        window='hann', # default, as specified by the documentation
        nperseg=window_size,
        noverlap=window_size // 2
    )
    
    utterances = []
    with open('../tsvs/%s.tsv' % video_id, 'rb') as f:
        for line in f:
            cols = [s.decode('utf-8') for s in line.rstrip(b'\n').split(b'\t')]
            utterances.append({
                'start': float(cols[0]),
                'end': float(cols[1]),
                'duration': float(cols[2]),
                'content': cols[3],
            })
    
    _num_examples, is_talking = labelsFromUtterances(
        utterances, 
        windows_per_second, 
        spectro.T.shape[0]
    )
    is_talking = is_talking[:-1].copy()
    was_talking = np.hstack(([0], is_talking[:-1]))
    was_was_talking = np.hstack(([0, 0], is_talking[:-2]))
    
    # TODO understand stft input and output shapes
    # Drop the last frame because I don't know how it's derived...
    return {
        'file_name': video_id,
        'signal_rate': rate,
        'window_size': window_size,
        'step_size': step_size,
        'num_utterances': len(utterances),
        'data': data, # TODO remove this line
        'freqs_vec': spectro.T[:-1, :-1],
        'is_talking': is_talking,
        'was_talking': was_talking,
        'was_was_talking': was_was_talking,
        # TODO phoneme
    }


def dict2packed(data: Dict[str, Any]) -> pd.DataFrame:
    num_rows = data['freqs_vec'].shape[0]
    step_size = data['step_size']  # type: int
    window_size = data['window_size']  # type: int
    frames = []
    for i in range(0, data['data'].shape[0], step_size):
        # Cast to lists because pandas doesn't allow numpy.ndarray in cells
        frames.append(list(data['data'][i : i + window_size]))
    return pd.DataFrame(data={
        'file_name': [data['file_name']] * num_rows,
        'signal_rate': [data['signal_rate']] * num_rows,
        'window_size': [window_size] * num_rows,
        'step_size': [step_size] * num_rows,
        'num_utterances': [data['num_utterances']] * num_rows,
        'raw_signal_vec': frames,
        'freqs_vec': data['freqs_vec'].tolist(),
        'is_talking': data['is_talking'],
        'was_talking': data['was_talking'],
        'was_was_talking': data['was_was_talking'],
        # TODO phoneme
    })


def packed2unpacked(df: pd.DataFrame) -> pd.DataFrame:
    assert df.shape[0] > 0, 'at least one row is required'
    data = {
        'file_name': df['file_name'],
        'signal_rate': df['signal_rate'],
        'window_size': df['window_size'],
        'step_size': df['step_size'],
        'num_utterances': df['num_utterances'],
        'is_talking': df['is_talking'],
        'was_talking': df['was_talking'],
        'was_was_talking': df['was_was_talking'],
    }
    window_size = df['window_size'].iat[0]
    for i in range(window_size):
        # The ternary operator here shouldn't be necessary. For some reason, the
        # last frame has half the signal size compared to all the other rows.
        data['raw_signal_vec[%d]' % i] = df['raw_signal_vec'].apply(
            lambda vec: vec[i] if i < len(vec) else None
        )
    step_size = df['step_size'].iat[0]
    # TODO understand stft input and output shapes
    # Why is frequencies limited to 240 complex values instead of 480?...
    for i in range(step_size):
        data['freqs_vec[%d]' % i] = df['freqs_vec'].apply(lambda vec: vec[i])
    return pd.DataFrame(data=data)


def utterancesFromPredictions(min_word_width: int, windows_per_second: int, predictions: np.ndarray) -> List[Dict[str, Any]]:
    ones = np.ones(min_word_width)
    i = 0
    predicted_utterances = []
    while i < predictions.shape[0] - min_word_width + 1:
        # 0 means ~"no words are spoken"
        if predictions[i] == 0:
            i += 1
            continue
        if (predictions[i : i + min_word_width] != ones).all():
            predictions[i] = 0
            i += 1
            continue
        for j in range(i + min_word_width, predictions.shape[0]):
            if predictions[j] == 0:
                j -= 1
                break
        predicted_utterances.append({
            'start': i / windows_per_second,
            'end': j / windows_per_second,
            'duration': (j - i) / windows_per_second,
            # content is TBD!
        })
        i = j + 1
    return predicted_utterances


def normalizeFreqs(freqs: np.ndarray, n_buckets: int) -> np.ndarray:
    # TODO scale dtype according to n_buckets
    assert n_buckets < 250, 'limit n_buckets to fit in int8'
    ret = np.zeros(freqs.shape, dtype=np.int8)
    buckets = list(map(lambda n: n / n_buckets, range(1, n_buckets)))
    # Take the transpose so each row represents the quantile summaries for
    # each column in freqs
    quantiles = np.quantile(freqs, buckets, axis=0).T
    # Enumerate each column of input
    # TODO vectorize this...
    for i in range(freqs.shape[1]):
        ret[:, i] = np.searchsorted(quantiles[i], freqs[:, i])
    return ret


# Viewing options:
# 1) Signal amplitude
# 2) Test signal amplitude (examples: sum(freqs[:5]), sum(freqs[5:10]), ...)
# 3) Spectrogram pcolormesh
# 4) IPython.display.Audio
# 5) JSON utterance labels
# 6) Time series labels, i.e. for (1-3), `plt.axvline(x=item['start'], color='#d62728')`, TODO: linewidth=wut?
# `%matplotlib notebook` may be handy?

# Phoneme Labeler:
# For each utterance, view a 3 second window.

# want: frames (aka windows) of 10 ms, steps of 5 ms.
# This is an example packed datastructure
# datastruct: (file_name, frame index, signal_rate (example: 44.1kHz), raw_signal_vec, freqs_vec (further want: speech_vec + background_vec), label, phoneme)
example_df = pd.DataFrame(data={
    'file_name': ['a', 'a', 'a', 'b', 'b'],
    'signal_rate': [44100, 44100, 44100, 44100, 44100],
    'window_size': [441, 441, 441, 441, 441],
    'step_size': [220, 220, 220, 220, 220],
    'frame_index': [0, 1, 2, 0, 1],
    'window_max_i': [0, 0, 1, 0, 1],
    # These are a bit misleading because their length is 2, but window_size says
    # they should be 441.
    'raw_signal_vec': [[0, 0], [1, 1], [1, 2], [0, 0], [0, 1]],
    'freqs_vec': [[0, 0], [1, 0], [2, 1], [0, 0], [0, 1]],
    # TODO: fft(fft(raw_signal)) b/c harmonics. consider librosa's "pitch class"
    'is_talking': [0, 0, 1, None, None],
    'was_talking': [0, 0, 0, 1, None],
    'was_was_talking': [0, 0, 0, 0, 1],
    'phoneme': [None, None, 'a', None, None],
})
example_df

In [ ]:
# readData gives a raw Dict/struct
# dict2packed repeats some data, like file_name, to fit into a DataFrame
# packed2unpacked makes a separate column for each field.
#df = dict2packed(readData(list(training_files)[0]))
df = pd.concat([dict2packed(readData(in_file, 60)) for in_file in list(training_files)[:2]])
df

In [ ]:
def showFreqsDistMatrix(freqs_vec: pd.Series) -> None:
    # This is how REPET-sim does it
    # `norm_freqs_vec = freqs_vec * (1.0 / np.sqrt(np.power(freqs_vec, 2).sum()))`
    # `true_divide` means `/` instead of `//`
    # `[:, np.newaxis]` adds a new dimension to the shape to allow for broadcasting to work
    # out and where are to avoid dividing by zero
    freqs_vec = np.abs(np.asarray(freqs_vec.tolist()))
    lengths = np.linalg.norm(freqs_vec, axis=1)
    norm_freqs_vec = np.true_divide(
        freqs_vec,
        lengths[:, np.newaxis],
        out=np.zeros(freqs_vec.shape, dtype='float64'),
        where=(lengths != 0.0)[:, np.newaxis]
    )
    # Contract: `np.power(norm_freqs_vec[i], 2).sum()` ~== 1, for all values of i.
    # Except for values of i that represent a zero vector. See above TODO

    sim_mat = np.matmul(norm_freqs_vec, norm_freqs_vec.T)
    diff_mat = np.subtract(1.0, sim_mat)

    frame_ids = np.arange(diff_mat.shape[0])
    plt.pcolormesh(frame_ids, frame_ids, diff_mat)
    plt.colorbar()
    plt.gcf().set_size_inches([15, 12]) # default is 6 x 4
    plt.show()
    
    # quantiles = np.percentile(diff_mat, list(range(5, 100, 5)))
    # near_counts = np.zeros((diff_mat.shape[0]))
    # for i in range(diff_mat.shape[0]):
    #   for j in range(diff_mat.shape[1]):
    #     if diff_mat[i, j] >= quantiles[0]:
    #       continue
    #     near_counts[i] += 1
    # np.percentile(near_counts, list(range(1, 100, 1)))
    
    return diff_mat


# diff_mat = showFreqsDistMatrix(df.freqs_vec)

In [ ]:
# Uncomment if you want to play with unpacked data...
# unpacked_df = packed2unpacked(df)
# cols_to_drop = [col for col in unpacked_df.columns.tolist() if col.startswith('raw_signal_vec')]

In [ ]:
cols = ['is_talking', 'was_talking', 'was_was_talking']
df[cols].groupby(cols).size().to_frame('num_frames')

In [ ]:
normalized = normalizeFreqs(np.abs(np.asarray(df.freqs_vec.tolist())), 20)

classifier = sklearn.tree.DecisionTreeClassifier(
    random_state=random.randint(0, 2 ** 32 - 1),
    max_depth=5
)
# `[:, :60]` means take all rows, but only the first 60 columns
combined = np.hstack([
    df.was_was_talking.to_numpy().reshape(normalized.shape[0], 1),
    df.was_talking.to_numpy().reshape(normalized.shape[0], 1),
    normalized[:, :60],
])
model = classifier.fit(combined, df.is_talking)
plt.gcf().set_size_inches([48, 16]) # default is 6 x 4
sklearn.tree.plot_tree(model, max_depth=3, node_ids=True)

In [ ]:
eval_df = pd.concat([dict2packed(readData(in_file, 60)) for in_file in list(eval_files)[:1]])
eval_normalized = normalizeFreqs(np.abs(np.asarray(eval_df.freqs_vec.tolist())), 20)

predictions = np.zeros(eval_normalized.shape[0], dtype='int8')
for i in range(eval_normalized.shape[0]):
    if i < 2:
        predictions[i] = 0
        continue
    predictions[i] = model.predict([np.hstack([
        predictions[i-2],
        predictions[i-1],
        eval_normalized[i, :60],
    ])])

# Compare predictions vs eval_df.is_talking
comparison = pd.concat([eval_df.is_talking, pd.Series(predictions, name='predictions')], axis=1)
comp_counts = comparison.groupby(['is_talking', 'predictions']).size()
print('Predicted %d windows of talking, but expected %d' % (comp_counts.loc[:, 1].sum(), comp_counts.loc[1, :].sum()))
print('Predicted %d windows of silence, but expected %d' % (comp_counts.loc[:, 0].sum(), comp_counts.loc[0, :].sum()))

# Any prediction shorter than this will be considered noise
min_word_width = 8
#predictions[:min_word_width] = np.zeros(min_word_width)
predicted_utterances = utterancesFromPredictions(
    min_word_width,
    eval_df.signal_rate.iat[0] / eval_df.step_size.iat[0],
    predictions
)
print('Predicted %d utterances, but actually had %d' % (
    len(predicted_utterances),
    eval_df.num_utterances.iat[0]
))

In [ ]:
comparison.groupby(['is_talking', 'predictions']).size().to_frame('num_windows')